In [68]:
import subprocess
import re
from datetime import datetime, timedelta
import pandas as pd

start_date_str = '2020-01-01'
end_date_str = '2020-02-01'

start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

date_list = []
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date.strftime('%Y-%m-%d'))
    current_date += timedelta(days=1)


In [69]:
def dbt_run(command:list[str], date_list:list[str]) -> pd.DataFrame:
    rows = []

    for i in range(len(date_list)-1):
        result = subprocess.run(['dbt', 
                                'run', 
                                '--select', 
                                'dim_customer_daily_snapshot', 
                                '--vars', 
                                f'{{start_date: {date_list[i]}, end_date: {date_list[i+1]}}}'],
                                shell=True, 
                                text=True,
                                stdout=subprocess.PIPE, 
                                universal_newlines=True,
                                cwd='./dim_model_scd_test')
        
        stats = re.search(r"\[\x1b\[32mSUCCESS (\d+)\x1b\[0m in (\d+\.\d+)s\]", result.stdout)
        row_count = int(stats[1])
        seconds = float(stats[2])

        rows.append({'start_date': date_list[i], 'end_date':date_list[i+1], 'row_count':row_count, 'seconds':seconds})

In [66]:
# daily snapshot
rows = []

for i in range(len(date_list)-1):
    result = subprocess.run(['dbt', 
                            'run', 
                            '--select', 
                            'dim_customer_daily_snapshot', 
                            '--vars', 
                            f'{{start_date: {date_list[i]}, end_date: {date_list[i+1]}}}'],
                            shell=True, 
                            text=True,
                            stdout=subprocess.PIPE, 
                            universal_newlines=True,
                            cwd='./dim_model_scd_test')
    
    stats = re.search(r"\[\x1b\[32mSUCCESS (\d+)\x1b\[0m in (\d+\.\d+)s\]", result.stdout)
    row_count = int(stats[1])
    seconds = float(stats[2])

    rows.append({'start_date': date_list[i], 'end_date':date_list[i+1], 'row_count':row_count, 'seconds':seconds})


In [67]:
rows

[{'start_date': '2020-01-01',
  'end_date': '2020-01-02',
  'row_count': 1,
  'seconds': 2.84},
 {'start_date': '2020-01-02',
  'end_date': '2020-01-03',
  'row_count': 556,
  'seconds': 3.5},
 {'start_date': '2020-01-03',
  'end_date': '2020-01-04',
  'row_count': 839,
  'seconds': 2.07}]